In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os, sys
# ──────────────────────── Path Configuration ───────────────────────
MainFolder      = Path("/mnt/DATA3_SSD/Minsoo")
# ──────────────────────── Parameters ────────────────────────
sys.path.append(Path(MainFolder) /"LDC")

# Note

In [ ]:
# ──────────────────────────────────────────────────────────────
# ⚙️ LDC Configuration Notes (Minsoo's Custom Settings)
# ──────────────────────────────────────────────────────────────

# 1. [main.py] Modify the input image directory
# Replace the default value of --input_val_dir:
#   parser.add_argument('--input_val_dir',
#                       type=str,
#                       default='/mnt/DATA3_SSD/Minsoo/Images3/')
# → Set this to your own folder containing test images.

# 2. [main.py] Set the output directory for results
#   parser.add_argument('--output_dir',
#                       type=str,
#                       default='/mnt/DATA3_SSD/Minsoo/LDC_Results')
# → Choose a different folder if needed (e.g., for experiment versioning).

# 3. [main.py] Match image dimensions to your dataset
#   parser.add_argument('--img_width', type=int, default=1080)
#   parser.add_argument('--img_height', type=int, default=710)
# → Make sure these match the resolution of your input images to avoid resizing errors.

# 4. [img_processing.py] Set custom threshold for edge detection output
# Look for this line:
#   tensor = np.where(tensor >= 0.70, tensor, 0)  # thresholding
# → Change the 0.70 to another value (e.g., 0.5 or 0.9) if you want to adjust the LDC edge sensitivity.

In [ ]:
!python "$MainFolder/LDC/main.py" --choose_test_data=-1

In [ ]:
image_dir       = MainFolder / "Images3"
csv_dir         = MainFolder / "csv3"
output_dir      = MainFolder / "image3csv3"
angle_csv_path  = MainFolder / "angles_from_hough.csv"
output_dir.mkdir(exist_ok=True)


margin_ratio = 0.05  

# ──────────────────────── Processing Loop ────────────────────────
angle_records = []

for image_path in sorted(image_dir.glob("*.png")):
    frame_name = image_path.name
    csv_path   = csv_dir / f"f_['{frame_name}'].csv"

    if not csv_path.exists():
        print(f"❌ CSV not found for {frame_name}")
        continue

    # Load image and edge map
    image_bgr = cv2.imread(str(image_path))
    if image_bgr is None:
        print(f"❌ Failed to load image: {image_path}")
        continue

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    edges     = pd.read_csv(csv_path, header=None).values.astype(np.uint8)

    # Hough Line Detection
    lines = cv2.HoughLinesP(
        edges, rho=1, theta=np.pi / 180, threshold=50,
        minLineLength=50, maxLineGap=10
    )

    height, width = image_rgb.shape[:2]
    top_margin    = int(height * margin_ratio)
    bottom_margin = height - top_margin

    max_length = 0
    longest_line = None

    if lines is not None:
        for (x1, y1, x2, y2) in lines[:, 0]:
            # Skip lines completely within top/bottom margins
            if (y1 < top_margin and y2 < top_margin) or \
               (y1 > bottom_margin and y2 > bottom_margin):
                continue

            length = np.hypot(x2 - x1, y2 - y1)
            if length > max_length:
                max_length = length
                longest_line = (x1, y1, x2, y2)

    # Visualization
    plt.figure(figsize=(10, 8))
    plt.imshow(image_rgb)
    if longest_line is not None:
        x1, y1, x2, y2 = longest_line
        plt.plot([x1, x2], [y1, y2], color='lightgreen', linewidth=2)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_dir / frame_name, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Angle Calculation
    if longest_line is not None:
        dx, dy = x2 - x1, y2 - y1
        theta_deg = (np.degrees(np.arctan2(-dy, dx)) + 180) % 180  # Range: 0–180°
        acute_angle = theta_deg if theta_deg <= 90 else 180 - theta_deg
    else:
        acute_angle = np.nan

    angle_records.append({"Frame": frame_name, "Angle_deg": acute_angle})

# Save Angle CSV
pd.DataFrame(angle_records).to_csv(angle_csv_path, index=False)
print(f"✅ Saved angle CSV to → {angle_csv_path}")
